# Introduction to Partial Differential Equations
---

## Chapter 4: Hyperbolic PDEs and the Wave Equation
---


## Creative Commons License Information
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/80x15.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title">Introduction to Partial Differential Equations: Theory and Computations</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" property="cc:attributionName" rel="cc:attributionURL">Troy Butler</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations" rel="dct:source">https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations</a>.

---
## Section 4.4: A Finite Difference Approximation
---

We approach this similarl to how we approached the discretization of the heat equation.

Specifically, we first perform a semidiscrete discretization (i.e., a method of lines discretization as seen in [Section 3.4](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp3/Chp3Sec4.ipynb), where we first apply the usual discretization in space to $-u_{xx}$ to arrive at
<br><br>
$$
    v_{tt} = -Av,
$$

where 

$$
A = \begin{pmatrix}
        2 & -1 & 0 & \cdots & 0 \\
        -1 & 2 & -1 & \ddots & \vdots \\
        0 & \ddots & \ddots & \ddots & 0 \\
        \vdots & \ddots & -1 & 2 & -1 \\
        0 & \cdots & 0 & -1 & 2
    \end{pmatrix},
$$

and the negative sign in front of $A$ in the semidiscrete approximation is due to the fact that our wave equation is based on $u_{xx}$ instead of $-u_{xx}$. As with the heat equation, $v$ is the $n$-dimensional vector that approximates the continuous solution $u$ at the interior grid points (interior because of the Dirichlet boundary conditions). Unlike our previous studies of the heat equation, we now have $v_{tt}$ instead of $v_t$. 

---
### Section 4.4.1 We use a centered finite difference approximation in time to approximate $v_{tt}$ at a grid point $x_j$ as
---

$$
    v_{tt}(x_j) \approx \frac{v_j^{m-1} - 2v_{j}^m + v_j^{m+1}}{(\Delta t)^2}.
$$

The ultimate question then becomes whether or not we define the fully discrete problem by setting this equal to the $j$th row of $-Av^{m}$ (for an explicit method) or the $j$th row of $-Av^{m+1}$ (for an implicit method).
Alternatively, we can write everything in matrix-vector format. 

For the explicit method, we have
<br><br>
$$
   \frac{v^{m-1} - 2v^m + v^{m+1}}{(\Delta t)^2} = -Av^m 
$$
<br><br>
and we solve for $v^{m+1}$ to get
<br><br>
$$
    v^{m+1} = (2I-(\Delta t)^2A)v^m - v^{m-1}. 
$$

A detail we have neglected is that applying this equation requires us to use both $v^m$ and $v^{m-1}$.
Thus, we cannot apply this at $m=0$ (which would imply we are solving for $v^1$). 
Somehow, we must specify $v^1$. 
This is done using a Taylor expansion in time.
Specifically, 
<br><br>
$$
    u(x,\Delta t) = u(x,0) + \Delta t u_t(x,0) + \frac{(\Delta t)^2}{2} u_{tt}(x,0) + O((\Delta t)^3), 
$$
<br><br>
and we make the substitutions $u(x,0)=f(x)$, $u_t(x,0) = g(x)$, and $u_{tt}(x,0) = u_{xx}(x,0)$ to get
<br><br>
$$
    u(x,\Delta t) = f(x) + \Delta t g(x) + \frac{(\Delta t)^2}{2} u_{xx}(x,0) + O((\Delta t)^3). 
$$

We therefore specify $v^1 \approx u(x,\Delta t)$ using the initial data as
<br><br>
$$
    v^1 = v^0 + \Delta t \left[\begin{array}{l} g(x_1) \\ g(x_2) \\ \vdots \\ g(x_n) \end{array}\right] - \frac{(\Delta t)^2}{2}Av^0.
$$

In [ ]:
def f(x):
    return x*(1-x)

def g(x):
    return 0*x

In [ ]:
def make_A(n):
    A = np.zeros((n,n))
    np.fill_diagonal(A,2)
    A += np.diag(-np.ones(n-1),k=1)
    A += np.diag(-np.ones(n-1),k=-1)    
    A *= 1/dx**2
    return A

In [ ]:
# Define and plot results for an explicit method

%matplotlib widget

n_s = [19, 39, 99, 199]
errors = np.zeros(4)
dt_s = np.zeros(4)
dx_s = np.zeros(4)
count = 0

for n in n_s:
    
    plt.figure(count)
    
    x = np.linspace(0,1,n+2)
    dx = x[1]-x[0]
    dx_s[count] = dx
    
    A = make_A(n)

    r = 1.0  # If this is greater than 1, then numerical solutions become unstable
    dt = r*dx
    dt_s[count] = dt
    
    v_old = f(x)
    v_current = 0*x
    v_current[1:-1] = v_old[1:-1] + dt*g(x[1:-1]) - dt**2/2.*np.dot(A,v_old[1:-1])
    
    v_new = 0*x
    
    t = 1*dt  # <----- start at 1*dt

    # bdry_force = np.zeros(n)

    B = 2*np.eye(n)-dt**2*A
    
    time_iter = 0
    while t<=10:
        v_new[1:-1] = np.dot(B,v_current[1:-1]) - v_old[1:-1]  # <--------- v_{m+1} is now computed
        v_old = np.copy(v_current)
        v_current = np.copy(v_new)
        t += dt  # <-------------- now go to t_{m+1}
        time_iter += 1
        
        if time_iter%5 == 0:
            plt.plot(x, v_new)
    
    plt.plot(x, v_new)
    plt.show()
            
    count += 1

<mark> **Making movies by exploiting pass-by-reference and in-place operators** <mark>

In [ ]:
def f(x):  # It is kind of fun to change this
    # Try x*(1-x)
    # Try x*(1-x)*(0.2-x)
    # Try np.sin(k * np.pi * x) for different k
    # Try x*np.sin(1/x)*(1-x)
    # Can also try some that don't satisfy the BCs like np.sin(1/x)
    # return x*np.sin(1/x)*(1-x)
    return x*(1-x)*(0.2-x)
    

In [ ]:
# Choose an n for discretizing space (and subsequently time)
n = 99  

# Now setup the spatial-temporal discretization
x = np.linspace(0,1,n+2)
dx = x[1]-x[0]
A = make_A(n)

r = 1.0  # If this is greater than 1, then numerical solutions become unstable
dt = r*dx

B = 2*np.eye(n)-dt**2*A

In [ ]:
# Now setup the numpy arrays that we utilize for visualization purposes
v_old = f(x)
v_current = 0*x
v_current[1:-1] = v_old[1:-1] + dt*g(x[1:-1]) - dt**2/2.*np.dot(A,v_old[1:-1])

def v_at_frame(v_current, v_old):
    # Create the solution to plot in a given frame of the movie
    v_new = np.dot(B,v_current[1:-1]) - v_old[1:-1] 
    
    # Update state of wave in-place
    v_old *= 0
    v_old += v_current
    v_current *= 0
    v_current[1:-1] += v_new
    return

In [ ]:
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

In [ ]:
fig, ax = plt.subplots()
line1 = ax.plot([], [], 'b')[0]  # Using b-. or b: for line-style will create a weird visual
line2 = ax.plot([], [], 'r:')[0]
line3 = ax.plot([], [], markersize=10, marker='s', mfc='k')[0]

ax.set_xlim(0, 1)
ax.set_ylim(-1.1*np.max(np.abs(f(x[1:-1]))), 1.1*np.max(np.abs(f(x[1:-1]))))

plt.title('The Wave')
time_text = ax.text(0.1, 0.9*np.max(np.abs(f(x[1:-1]))), "", 
                    fontsize=15, color='red',
                    bbox=dict(facecolor='blue', alpha=0.1))
plt.close()

def animate_v(frame, v_current, v_old):
    v_at_frame(v_current, v_old)
    line1.set_data((x, v_current))
    t = frame*dt
    time_text.set_text("Time: {:5.2f}".format(t))
    line2.set_data([0.5, 0.5], [-1, 1])
    marker_idx = np.argmax(np.abs(v_current))
    #line3.set_data(x[marker_idx],v_current[marker_idx])
    return line1, line2, line3

anim = FuncAnimation(fig, animate_v, frames=int((4/dt)), fargs=(v_current, v_old,), interval=1000*dt)

In [ ]:
HTML(anim.to_jshtml())

---
### Section 4.4.2: Stability Analysis
---

In the code cells above, notice the comment about the `r` value used:

```# If this is greater than 1, then numerical solutions become unstable```

We now explore what is meant by that comment.

First, we observe that

$$
    u_k(x,t) = \sin(k\pi x)e^{\pm ik\pi t}
$$

satisfies the wave equation where we have used Euler's identity to represent the $T_k(t)$ functions. 

<mark>We now study the relationship between $\sin(k\pi x)$ and solutions to the discrete problem with a little help from [Lemma 2.5.6 in Section 2.5.](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp2/Chp2Sec5.ipynb)</mark>

For each $1\leq k\leq n$, we have that

- The vector $X_k\in\mathbb{R}^n$ defined by $X_{k,j}=\sin(k\pi x_j)$ (i.e., the $j$th component is given by evaluating $\sin(k\pi x)$ at the grid point $x_j$) is an eigenvector of the matrix $A$ that we are familiar with from our usual spatial finite difference approximation. 

- The associated eigenvalues are given by $\mu_k = \frac{4}{(\Delta x)^2}\sin^2(k\pi \Delta x/2)$. 

With this in mind, we make an ansatz that for each $1\leq k\leq n$ there exists some nonzero $a\in\mathbb{C}$ such that 

$$v^m = X_ka^m$$

is a solution to the finite difference equation for the wave equation:

$$
    v^{m+1} = (2I-(\Delta t)^2A)v^m - v^{m-1}. 
$$

Inserting the ansatz into this finite difference equation and using some algebra, we find that $a$ must satisfy the following equation

$$
    a^2 -(2-s)a +1 = 0, 
$$

where, 

$$
    s=(\Delta t)^2 \mu_k = 4\left(\frac{\Delta t}{\Delta x}\right)^2 \sin(k\pi \Delta x/2).
$$

Since the exact particular solutions have the property that $\vert u_k(x,t)\vert\leq 1$ for all $k\in\mathbb{Z}$, $0\leq x\leq 1$ and $t\geq 0$, we need to ensure that $\vert a \vert\leq 1$.

By the quadratic formula, the roots of $a^2-(2-s)a+1$ are given by

$$
    \frac{1}{2}\left[(2-s)\pm \sqrt{(2-s)^2-4}\right]
$$

<mark>**Summary to this point:**</mark>

- We need $\vert a\vert\leq 1$. 

- $a$ depends on $s$.

- $s$ depends upon the ratio $r=\frac{\Delta t}{\Delta x}$. 

<mark>**Strategy:**</mark>

- We examine the (complex-valued) roots of $a$ function of $s$ in the code cell below.

- This provide us insight into what values of $s$ are ***permissible*** in the sense that they produce ***stable*** solutions. 

- This in turn produces restrictions on the values of $r$.

- We use restrictions on $r$ to determine appropriate values of $\Delta t$ given a particular choice of spatial discretization, and now the code comments make sense.

In [ ]:
%matplotlib widget

s = np.linspace(-3, 7, 101).astype('complex')  # Necessary for the square root operation below

plt.figure(4)
plt.plot(s, np.abs(0.5*(2-s-np.sqrt((2-s)**2-4))))
plt.plot(s, np.abs(0.5*(2-s+np.sqrt((2-s)**2-4))))
plt.show()

<mark>**Observations from above plot:**</mark>

- For $s<0$ or $s>4$, $a$ has two real roots with one having a magnitude greater than $1$, so we better avoid these roots.

- At $s=0$, there is a double real root of $a=1$. This also implies $\Delta t=0$, which is just silly. The numerical solutions would also just be $v^m = X_k$ for all $m$, and a constant vector is hardly going to produce any type of "wave behavior" that we want. We are left with considering $s>0$ in practice.

- At $s=4$, there is a double real root of $a=-1$. This implies $\Delta t=\Delta x$ and it results in numerical solutions $v^m = X_k (-1)^m$ for all $m$ that will oscillate. So, these seem okay, and we can let $s\leq 4$.

In conclusion, we need $s\in (0,4]$, and by examining what this implies about $r$, we need $r\leq 1$ to achieve this.

---
## Navigation:

- [Previous](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp4/Chp4Sec3.ipynb)

- [Next](https://github.com/CU-Denver-MathStats-OER/Intro-PDEs-Theory-and-Computations/blob/main/Chp4/Chp4Sec5.ipynb)
---